In [1]:
import warnings

import joblib
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from tqdm import tqdm

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [2]:
path = "./data/preprocessing"

busan_spot_info = pd.read_csv("./data/preprocessing/busan_spot_info - final.csv", encoding="cp949")
travel = pd.read_csv("./data/preprocessing/travel.csv")
traveller_master = pd.read_csv("./data/preprocessing/traveller_master.csv")

In [3]:
def mission_check(travel):
    travel_list = []
    for i in range(len(travel)):
        value = int(travel['TRAVEL_MISSION_CHECK'][i].split(';')[0])
        travel_list.append(value)
    return travel_list


travel['TRAVEL_MISSION_PRIORITY'] = mission_check(travel)
travel = travel[['TRAVEL_ID', 'TRAVELER_ID', 'TRAVEL_MISSION_PRIORITY', 'MVMN_NM']]

In [4]:
# 캠핑 삭제
# 스파/온천/성수 준비 삭제
# 반려동물 동반 여행 삭제
# 친환경 여행 삭제
# 등반 여행 삭제
exclude_priorities = [8, 10, 25, 27, 28]
travel = travel[~travel['TRAVEL_MISSION_PRIORITY'].isin(exclude_priorities)]

In [5]:
travel_style_columns = ['TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 
                        'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8']


traveller_master['TRAVEL_STYL'] = traveller_master[travel_style_columns].mode(axis=1)[0]


traveller_master = traveller_master[['TRAVELER_ID', 'GENDER', 'AGE_GRP', 'INCOME', 'TRAVEL_STYL', 
                                      'TRAVEL_MOTIVE_1', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM' ]]

In [6]:
busan_spot_info

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,ROAD_NM_CD,LOTNO_CD,POI_ID,POI_NM,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,SIDO,GUNGU
0,2209040011,b_b001138,7,광안리해수욕장,2022-09-04,2022-09-04,부산 수영구 광안해변로 219,부산 수영구 광안동 192-20,129.118976,35.153193,NaN,NaN,POI01000000BZ0BJ0,광안리해수욕장,30,1,N,1.0,NaN,4.0,4.0,4.0,NaN,부산,수영구
1,2209070015,b_b002682,14,센텀시티 스파랜드,2022-09-07,2022-09-07,부산 해운대구 센텀남대로 35,부산 해운대구 우동 1495,129.129078,35.168638,NaN,NaN,NaN,NaN,60,6,N,7.0,NaN,3.0,2.0,4.0,NaN,부산,해운대구
2,2210280006,b_b008221,5,F1963,2022-10-28,2022-10-28,부산 수영구 구락로123번길 20,부산 수영구 망미동 475-1,129.114938,35.176928,NaN,NaN,POI01000000B78RCP,고려제강F1963,90,4,Y,1.0,NaN,4.0,3.0,3.0,NaN,부산,수영구
3,2211030011,b_b010232,10,흰여울문화마을,2022-11-03,2022-11-03,NaN,부산 영도구 영선동4가 1044-6,129.045281,35.077915,NaN,NaN,NaN,NaN,30,7,N,1.0,NaN,5.0,5.0,5.0,NaN,부산,영도구
4,2208140004,b_b000618,18,베스트루이스해밀턴호텔 오션테라스,2022-08-14,2022-08-14,부산 기장군 기장읍 연화1길 27,부산 기장군 기장읍 연화리 376-4,129.222334,35.213825,NaN,NaN,POI010000006TE6M1,전망대횟집,150,6,N,2.0,NaN,3.0,2.0,3.0,NaN,부산,기장군
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3582,2309160004,f_f016095,3,광안리해수욕장,2023-09-16,2023-09-16,부산 수영구 광안해변로 219,부산 수영구 광안동 192-20,129.118976,35.153193,NaN,NaN,POI01000TR030726V,광안리해수욕장,30,8,Y,1.0,NaN,5.0,5.0,5.0,NaN,부산,수영구
3583,2309140005,f_f016595,4,마가 만두,2023-09-14,2023-09-14,부산 동구 대영로243번길 56,부산 동구 초량동 563,129.038514,35.114510,NaN,NaN,POI01000TR035419V,마가만두,30,4,N,5.0,NaN,4.0,4.0,4.0,NaN,부산,동구
3584,2309140008,f_f016595,6,국제시장,2023-09-14,2023-09-14,부산 중구 책방골목길 16,부산 중구 보수동1가 151-1,129.026657,35.103367,NaN,NaN,POI01000TR011320V,국제시장,60,4,N,1.0,NaN,5.0,4.0,4.0,NaN,부산,중구
3585,2309140010,f_f016595,7,국제시장,2023-09-14,2023-09-14,NaN,부산 중구 창선동2가 40-5,129.028890,35.099449,NaN,NaN,POI03000TR022903V,국제시장,60,4,Y,1.0,NaN,1.0,5.0,5.0,NaN,부산,중구


In [7]:
busan_spot_info = busan_spot_info[['TRAVEL_ID', 'VISIT_AREA_NM', 'SIDO', 'GUNGU', 'VISIT_AREA_TYPE_CD', 'DGSTFN',
                                  'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'RESIDENCE_TIME_MIN', 'REVISIT_YN',
                                  'X_COORD', 'Y_COORD']].reset_index(drop=True)

In [8]:
"""Age Group 40대 이상은 40으로 변경"""
traveller_master['AGE_GRP'] = traveller_master['AGE_GRP'].apply(lambda x: 40 if x >= 40 else x)

In [9]:
"""busan spot info와 traveller_master, travel merge Result: 부산에 여행온 사람들의 정보"""
df = pd.merge(travel, traveller_master, left_on = 'TRAVELER_ID', right_on = 'TRAVELER_ID', how = 'inner')
df = pd.merge(busan_spot_info, df, left_on = 'TRAVEL_ID', right_on = 'TRAVEL_ID', how = 'left')  

In [10]:
"""Preprocessing"""

df["RESIDENCE_TIME_MIN"] = df["RESIDENCE_TIME_MIN"].replace(0, df["RESIDENCE_TIME_MIN"].median())

df['REVISIT_YN'] = df['REVISIT_YN'].replace("N",0)
df['REVISIT_YN'] = df['REVISIT_YN'].replace("Y",1)

df = df.dropna(subset = ['TRAVEL_STYL'])
df = df.reset_index(drop= True)
df = df.dropna(subset = ['TRAVEL_MOTIVE_1'])
df = df.reset_index(drop= True)

In [11]:
"""특정 관광지에 방문한 사람들 중 랜덤으로 한 명을 선택하여 그 사람의 모든 여행 데이터를 새로운 데이터프레임 train_df에 추가합니다"""
df_copy = df.copy()
train_df = pd.DataFrame(columns=df.columns)  

for area in tqdm(df['VISIT_AREA_NM'].unique()):
    area_visitors_df = df_copy[df_copy['VISIT_AREA_NM'] == area]  # 특정 관광지에 방문한 모든 사람의 데이터를 추출
    
    np.random.seed(42)  # 랜덤 시드 설정
    if area_visitors_df.empty:  # 만약 해당 관광지에 방문한 사람이 없으면
        continue  # 아무 작업도 하지 않고 다음 반복으로 넘어감
    
    random_idx = np.random.randint(len(area_visitors_df))  # 해당 관광지에 방문한 사람 중 랜덤으로 한 명의 인덱스를 생성
    random_visitor_df = area_visitors_df.iloc[[random_idx]]  # 랜덤으로 선택된 사람의 데이터 추출
    travel_id = random_visitor_df.iloc[0, 0]  # 해당 사람의 'TRAVEL_ID' 값을 추출
    
    visitor_trips_df = df_copy[df_copy['TRAVEL_ID'] == travel_id]  # 선택된 사람의 모든 여행 데이터를 추출
    
    df_copy = pd.merge(visitor_trips_df, df_copy, how='outer', indicator=True)  # 원본 데이터프레임과 선택된 사람의 데이터를 병합, 병합 결과를 표시
    df_copy = df_copy.query('_merge == "right_only"').drop(columns=['_merge'])  # 원본 데이터프레임에서 선택된 사람의 데이터를 삭제
    
    train_df = train_df.append(visitor_trips_df, ignore_index=True)  # 선택된 사람의 데이터를 train_df 데이터프레임에 추가

100%|██████████| 649/649 [00:05<00:00, 118.98it/s]


In [12]:
while len(df_copy) / len(df) > 0.15:
    np.random.seed(42)  # 랜덤 시드 설정
    random_idx = np.random.randint(len(df_copy))  # 랜덤 인덱스 생성
    random_visitor = df_copy.iloc[[random_idx]]  # 랜덤으로 선택된 사람의 데이터 추출
    travel_id = random_visitor.iloc[0, 0]  # 해당 사람의 'TRAVEL_ID' 값 추출
    
    visitor_trips = df_copy[df_copy['TRAVEL_ID'] == travel_id]  # 선택된 사람의 모든 여행 데이터 추출

    df_copy = pd.merge(visitor_trips, df_copy, how='outer', indicator=True)  # 원본 데이터프레임과 선택된 사람의 데이터를 병합, 병합 결과를 표시
    df_copy = df_copy.query('_merge == "right_only"').drop(columns=['_merge'])  # 원본 데이터프레임에서 선택된 사람의 데이터를 삭제
    
    train_df = train_df.append(visitor_trips, ignore_index=True)  # 선택된 사람의 데이터를 train_df 데이터프레임에 추가

In [13]:
new_train = pd.DataFrame(columns = list(train_df.columns) + ['RESIDENCE_TIME_MIN_mean', 'RCMDTN_INTENTION_mean',
                                                          'REVISIT_YN_mean', 'TRAVEL_COMPANIONS_NUM_mean',
                                                          'REVISIT_INTENTION_mean'])


for i in tqdm(list(train_df['VISIT_AREA_NM'].unique())): #유니크한 관광지 목록 중에서
    df2 = train_df[train_df['VISIT_AREA_NM'] == i] # 특정 관광지에 간 모든 사람 뽑아서
    for j in ['RESIDENCE_TIME_MIN', 'RCMDTN_INTENTION', 'REVISIT_YN', 'TRAVEL_COMPANIONS_NUM', 'REVISIT_INTENTION']:
        #체류시간 평균 산출 
        globals()[str(j)+'_mean'] = df2[str(j)]
        globals()[str(j)+'_mean'] = np.mean(globals()[str(j)+'_mean'])
        #데이터프레임에 들어가게 값을 리스트 형태로 변환
        globals()[str(j)+'_mean'] = np.repeat(globals()[str(j)+'_mean'], len(df2)) 
        df2[str(j)+'_mean'] = globals()[str(j)+'_mean']
    #새로운 데이터프레임에 방문지별 평균값 대입
    new_train = pd.concat([new_train, df2], axis = 0)
    
new_train.sort_values(by = ['TRAVEL_ID'], axis = 0, inplace = True)

100%|██████████| 649/649 [00:00<00:00, 682.38it/s]


In [14]:
"""5번 이상 방문한 곳으로만 필터링"""
new_train.reset_index(drop = True, inplace= True)

count = pd.DataFrame(new_train['VISIT_AREA_NM'].value_counts())
count['places'] = count.index
five_places = list(count[count['VISIT_AREA_NM']>=5]['places']) 
for i in range(len(new_train)):
    if new_train['VISIT_AREA_NM'][i] not in five_places:
        new_train = new_train.drop([i], axis = 0)
new_train.reset_index(drop = True, inplace = True)

In [15]:
visit5 = new_train[['VISIT_AREA_NM', 'X_COORD', 'Y_COORD']]

In [16]:
from scipy import stats

value_counts = new_train['VISIT_AREA_NM'].value_counts().reset_index()
value_counts.columns = ['VISIT_AREA_NM', 'count']
gungu = new_train.groupby(['VISIT_AREA_NM', 'GUNGU']).size().reset_index(name='count')

def mode_coordinates(group):
    lat_mode = stats.mode(group['X_COORD'])[0]
    lon_mode = stats.mode(group['Y_COORD'])[0]
    return pd.Series([lat_mode, lon_mode], index=['X_COORD', 'Y_COORD'])

coordinates = new_train.groupby('VISIT_AREA_NM').apply(mode_coordinates).reset_index()
coordinates['GUNGU'] = gungu['GUNGU']

In [17]:
coordinates.to_csv(path + '/coordinates.csv', index=False)

In [18]:
import pandas as pd

# Data for the DataFrame
data = {
    'GUNGU': ['중구', '서구', '동구', '영도구', '부산진구', '동래구', '남구', '북구', '해운대구', '사하구', '금정구', '강서구', '연제구', '수영구', '사상구', '기장군'],
    'population': [38487, 103694, 87460, 106233, 360881, 269350, 255107, 272315, 378753, 296253, 214168, 141783, 210999, 173944, 202062, 177912]
}

# Creating the DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df.to_csv(path + '/population.csv', index=False)


In [19]:
df

,GUNGU,population
0,중구,38487
1,서구,103694
2,동구,87460
3,영도구,106233
4,부산진구,360881
5,동래구,269350
6,남구,255107
7,북구,272315
8,해운대구,378753
9,사하구,296253


In [20]:
coordinates = pd.merge(coordinates, df, on='GUNGU', how='left')
coordinates


,VISIT_AREA_NM,X_COORD,Y_COORD,GUNGU,population
0,BIFF 광장,129.029046,35.098868,중구,38487
1,F1963,129.114938,35.176928,수영구,173944
2,KT&G상상마당 부산,129.057299,35.154260,부산진구,360881
3,NC백화점 서면점,129.063253,35.157164,부산진구,360881
4,감천문화마을,129.009428,35.096300,사하구,296253
...,...,...,...,...,...
76,해운대 해수욕장,129.159855,35.158523,해운대구,378753
77,해운대블루라인파크 청사포정거장,129.191364,35.161216,해운대구,378753
78,행리단길,129.157909,35.165111,해운대구,378753
79,황령산,129.081975,35.157249,부산진구,360881


In [21]:
coordinates.to_csv(path + '/coordinates-final.csv', index=False)